# Fully finetuning a LLM for a new task using qlora and custom heads.
In the *text_classification_linear_probe* notebook we found that the output of the third to last block of a transformer trained on causal_lm has the most linearly seperable information on the sentiment of a sentence. To get even better performance on the sentiment prediction task, we can finetune the whole transformer instead of only the newly added head. In this example, we will jointly train a qlora on the body of the transformer while also fully training a newly initialized head for sentiment classification.

In [1]:
from transformer_heads import create_headed_qlora
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    MistralForCausalLM,
    Trainer,
    BitsAndBytesConfig,
    TrainingArguments,
    GPT2Model,
    GPT2LMHeadModel,
)
from transformer_heads.util.helpers import DataCollatorWithPadding, get_model_params
from peft import LoraConfig
from transformer_heads.config import HeadConfig
from transformer_heads.util.model import print_trainable_parameters
from transformer_heads.util.evaluate import (
    evaluate_head_wise,
    get_top_n_preds,
    get_some_preds,
)
import torch
import pandas as pd

In [ ]:
model_path = "gpt2"
train_epochs = 1
eval_epochs = 1
logging_steps = 100
train_batch_size = 4
eval_batch_size = 4

In [ ]:
model_params = get_model_params(model_path)
model_class = model_params["model_class"]
hidden_size = model_params["hidden_size"]
vocab_size = model_params["vocab_size"]
print(model_params)

Only one head for now, as we aim to maximize the performance of the head hooked at layer -3 (third to last).

In [3]:
head_configs = [
    HeadConfig(
        name=f"imdb_head_3",
        layer_hook=-3,
        in_size=hidden_size,
        output_activation="linear",
        pred_for_sequence=True,
        loss_fct="cross_entropy",
        num_outputs=2,
    )
]

In [4]:
dd = load_dataset("imdb")

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    out = tokenizer(examples["text"], padding=False, truncation=True)
    for hc in head_configs:
        out[hc.name] = examples["label"]
    return out


for split in dd.keys():
    dd[split] = dd[split].filter(function=lambda example: len(example["text"]) > 10)
    dd[split] = dd[split].shuffle()
    dd[split] = dd[split].map(tokenize_function, batched=True)

dd.set_format(
    type="torch",
    columns=["input_ids", "attention_mask"] + [x.name for x in head_configs],
)
for split in dd.keys():
    dd[split] = dd[split].remove_columns(["text", "label"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

We want to train a LoRA, so we have to define a LoRA config. The *create_headed_qlora* function will automatically set up our model with *requires_grad* True only for the LoRA parameters and for the parameters of the heads.

Setting *target_modules=None* in the qlora config will make *create_headed_qlora* create LoRA modules for all linear layers in the transformer.

In [6]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=None,
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
)
model = create_headed_qlora(
    base_model_class=model_class,
    model_name=model_path,
    quantization_config=quantization_config,
    lora_config=lora_config,
    head_configs=head_configs,
    fully_trained_heads=True,
    device_map={"": torch.cuda.current_device()},
)

Some weights of TransformerWithHeads were not initialized from the model checkpoint at gpt2 and are newly initialized: ['heads.imdb_head_3.lins.0.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
print_trainable_parameters(model)

all params: 91411200 || trainable params: 9438720 || trainable%: 10.325561856752783
params by dtype: defaultdict(<class 'int'>, {torch.float32: 48943872, torch.uint8: 42467328})
trainable params by dtype: defaultdict(<class 'int'>, {torch.float32: 9438720})


In [8]:
collator = DataCollatorWithPadding(
    feature_name_to_padding_value={
        "input_ids": tokenizer.pad_token_id,
        "attention_mask": 0,
    }
)

In [9]:
args = TrainingArguments(
    output_dir="imdb_linear_probe",
    learning_rate=0.0002,
    num_train_epochs=train_epochs,
    logging_steps=logging_steps,
    do_eval=False,
    remove_unused_columns=False,
    optim="paged_adamw_32bit",
    gradient_checkpointing=True,
    lr_scheduler_type="constant",
    ddp_find_unused_parameters=False,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
)
trainer = Trainer(
    model,
    args=args,
    train_dataset=dd["train"],
    data_collator=collator,
)
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ykeller (chm-hci). Use `wandb login --relogin` to force relogin


Step,Training Loss
20,2.109500
40,0.921800
60,0.882500
80,0.842900
100,0.876900
120,0.795600
140,0.718800
160,0.664800
180,0.503900
200,0.733200


Checkpoint destination directory imdb_linear_probe/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory imdb_linear_probe/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory imdb_linear_probe/checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory imdb_linear_probe/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory imdb_linear_probe/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory imdb_linear_probe/checkpoint-3000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=3125, training_loss=0.33326138354301454, metrics={'train_runtime': 862.0359, 'train_samples_per_second': 29.001, 'train_steps_per_second': 3.625, 'total_flos': 9479914418995200.0, 'train_loss': 0.33326138354301454, 'epoch': 1.0})

In [10]:
print(
    evaluate_head_wise(
        model, dd["test"], collator, epochs=eval_epochs, batch_size=eval_batch_size
    )
)

Evaluating: 100%|██████████| 3125/3125 [06:24<00:00,  8.13it/s]

(0.24573207373935263, {'imdb_head_3': 0.24573207373935263})


In [11]:
ins, preds, ground_truths = get_some_preds(
    model, dd["test"], tokenizer, n=20, classification=True
)
print(
    pd.DataFrame(
        list(zip(ins, preds["imdb_head_5"], ground_truths["imdb_head_5"])),
        columns=["review", "label", "ground_truth"],
    )
)

Predicting: 100%|██████████| 20/20 [00:00<00:00, 32.61it/s]


Empty DataFrame
Columns: [review, label]
Index: []
